In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import re

In [2]:
%run ../../../Util/FileFolder
%run ../../../Util/Variable
%run ../../../Util/ImageProcessing

created


In [5]:

f'{count_files_in_folder(ayush_classFication_Working)} images in The Dataset' # from File Handle

'48015 images in The Dataset'

In [6]:
volume=folder_size_in_gb(ayush_classFication_Working) # from File Handle
f'Dataset have  {volume} GB Data'

'Dataset have  153.97459274809808 GB Data'

In [3]:
create_new_folders(today_classfication_analysis_path)

In [9]:
reportjsonfile=os.path.join(today_classfication_analysis_path,'reportjsonfile.json')

In [10]:
with open(reportjsonfile, 'w', encoding='utf-8') as json_file:
    json.dump({
        'volume_gb':volume,
        'count':count_files_in_folder(ayush_classFication_Working)
        }, json_file, indent=2)

In [11]:
subfolder={}
subfolderCount=num_sub_folders(ayush_classFication_Working)
subfolder['count']=subfolderCount

In [12]:
folders_names=sub_folders(ayush_classFication_Working)

In [13]:
folders=list(
    map(
        lambda name: 
            {'name':name,
             'path':os.path.join(ayush_classFication_Working,name),
             'filecount':count_files_in_folder(os.path.join(ayush_classFication_Working,name)),
             'volume':{
                 'GB':folder_size_in_gb(os.path.join(ayush_classFication_Working,name)),
                 "KB":folder_size_in_kb(os.path.join(ayush_classFication_Working,name)),
                "MB":folder_size_in_mb(os.path.join(ayush_classFication_Working,name))
             }
            },folders_names))
folder_li=list(map( lambda name: 
            [name,
             folder_size_in_gb(os.path.join(ayush_classFication_Working,name)),
            folder_size_in_kb(os.path.join(ayush_classFication_Working,name)),
            folder_size_in_mb(os.path.join(ayush_classFication_Working,name))
             
            ],folders_names))
folder_count_li=list(map( lambda name: 
            [name,
            count_files_in_folder(os.path.join(ayush_classFication_Working,name)),
             
            ],folders_names))

In [14]:
save_to_csv_filedetails(data=folder_li,colums=['Names','VolumeKB','VolumeMB',"VolumeGB"],csv_filename='volumereportcsv.csv',save_folder=today_classfication_analysis_path)

32

In [15]:
save_to_csv_filedetails(data=folder_count_li,colums=['Names','counts'],csv_filename='countreportcsv.csv',save_folder=today_classfication_analysis_path)

32

In [16]:
subfolderpaths=subfolders_path(ayush_classFication_Working)

In [17]:
subfolders=[]
for subfolder in folders:
     subfoldernames=sub_folders(subfolder['path'])
     subchild=list(
        map(lambda name: {
                'name':name,
                'path':os.path.join(subfolder['path'],name),
                 'filecount':count_files_in_folder(os.path.join(subfolder['path'],name)),
                 'volume':{
                     'GB':folder_size_in_gb(os.path.join(subfolder['path'],name)),
                     "KB":folder_size_in_kb(os.path.join(subfolder['path'],name)),
                    "MB":folder_size_in_mb(os.path.join(subfolder['path'],name)),
                 },
                },subfoldernames)),
     subfolder['subfolders']=subchild,
     subfolders.append(subfolder)

In [18]:
leafpattern = re.compile(r'^(leaf|leafs|leaves|Leaves|Leaf)$')
plantpattern=re.compile(r'^(Plants|plant|plants|p|Plant)$')
rawFruitpattern=re.compile(r'^(fruits|Fruit|kaai|Fruits|fruit|raw_fruits|seeds|kai)$')
flowerpattern=re.compile(r'^(Flower|flower|flowers)$')
Rootpattern=re.compile(r'^(kilanku|Root|root)$')
subsub=[]
for i,subpath in enumerate(subfolderpaths):
    subsubnames=sub_folders(subpath)
    subsubpath=subfolders_path(subpath)
    sub={
        'speciesname':folders_names[i],
    }
    for ind,s in enumerate(subsubnames):
        sub['path']=subsubpath[ind]
        if leafpattern.match(s):
            sub['leaves']=count_files_in_folder(subsubpath[ind])
        elif plantpattern.match(s):
            sub['plants']=count_files_in_folder(subsubpath[ind])
        elif flowerpattern.match(s):
            sub['flowers']=count_files_in_folder(subsubpath[ind])
        elif rawFruitpattern.match(s):
            sub['raw_fruits']=count_files_in_folder(subsubpath[ind])
        elif Rootpattern.match(s):
            sub['root']=count_files_in_folder(subsubpath[ind])
        else:
             sub[s]=count_files_in_folder(subsubpath[ind])
    subsub.append(sub)

In [19]:
def subpartcountmaping(subpart):
    subpartRow=[]
    subpartRow.append(subpart['speciesname'])
    subpartRow.append(subpart['path'])
    for part in ['leaves','plants','flowers','raw_fruits','root']:
        subpartRow.append(subpart[part]) if part in subpart.keys() else subpartRow.append(0)
    return subpartRow

In [20]:
subsubli=list(map(lambda tr:subpartcountmaping(tr),subsub))

In [21]:
with open(reportjsonfile, 'r+') as f:
        # Load existing data from JSON
        existing_data = json.load(f)
        
        # Append new data to existing JSON data
        existing_data['subfolder']=subfolders
        
        # Write updated data back to JSON file
        f.seek(0)
        json.dump(existing_data, f, indent=4)
        f.truncate()

In [22]:
subfolderdetails =list(map(lambda folder: {
    'name':folder['name'],
    'path':folder['path'],
    'files':folder['filecount'],
    'volume(MB)':folder['volume']['MB'],
    },subfolders)) 
subfolderdetails_li=list(map(lambda folder: [
    folder['name'],
    folder['path'],
    folder['filecount'],
    folder['volume']['MB'],
    len(folder['subfolders'])
    ],subfolders)) 

In [23]:
save_to_csv_filedetails(data=subsubli,colums=["Speciesname","Path","Leaves","Plants","Flowers","rawFruits","Root"],csv_filename='subpart_reportcsv.csv',save_folder=today_classfication_analysis_path)

32

In [24]:
save_to_csv_filedetails(data=subfolderdetails_li,colums=["Names","Path","Files","Volume MB","subfolders"],csv_filename='reportcsv.csv',save_folder=today_classfication_analysis_path)

32